In [1]:
import pandas as pd
import numpy as np
import talib as talib
from pyecharts import options as opts
from pyecharts.globals import *
from pyecharts.charts import *
import re
# from pyecharts.

from datetime import datetime,timedelta
import matplotlib.pyplot as plt

g_colors = [
    '#111111',
    '#f06020',
    '#1ac6bc',
    '#cc1b9e',
    '#bde01e',
    '#211ac6',
    '#f0bd20',
    '#791ac6',
    '#f0e620',
    '#e61f2e',
    '#38cb1b',
    '#74d3ff',
    '#ffb174',
    '#678ce3',
    '#ffcd74',
    '#8867e3',
    '#ffe974',
    '#cf67e3',
    '#b9ee6c',
    '#ff7f74',
    '#ed6ca1',
    '#67e384',
    '#ff7f74'
]


# %matplotlib inline
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

#使用tushare获取交易数据
#设置token
import tushare as ts
#注意token更换为你在tushare网站上获取的
token='36d50a0386e9066b5c79c2de58ea2d3a9b980020041adc910f76bfc7'
ts.set_token(token)
pro=ts.pro_api()
index={'上证综指': '000001.SH',
        '深证成指': '399001.SZ',
        '沪深300': '000300.SH',
        '创业板指': '399006.SZ',
        '上证50': '000016.SH',
        '中证500': '000905.SH',
        '中小板指': '399005.SZ',
        '上证180': '000010.SH'}
#获取当前交易的股票代码和名称
def get_code():
    df = pro.stock_basic(exchange='', list_status='L')
    codes=df.ts_code.values
    names=df.name.values
    stock=dict(zip(names,codes))
    #合并指数和个股成一个字典
    stocks=dict(stock,**index)
    return stocks

def get_index_code():
    return index;

def get_code_df():
    df = pro.stock_basic(exchange='', list_status='L')
    return df

def get_ts_code ( keyword ): 
    code_df = get_code_df()
    code =''
    if re.match( r'\d{6}', keyword):
        code_df.index = code_df['symbol']
    else:
        code_df.index = code_df['name']
    code=code_df.loc[keyword,'ts_code']
    symbol=code_df.loc[keyword,'symbol']
    name=code_df.loc[keyword,'name']
    return (code,symbol,name)

/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [18]:
def get_line_income(df,stock_info) -> Line:
    trade_dates = np.array(df.ann_date).tolist()
    n_incomes =np.array(df.n_income_attr_p).tolist()
    operate_profits = np.array(df.operate_profit).tolist()
    
    c = (
        Line()
        .add_xaxis(trade_dates)
        .add_yaxis("净利润"
                  , n_incomes
                  , is_smooth=True,
                    is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.9),
            label_opts=opts.LabelOpts(is_show=False),
                  )
        .add_yaxis("营业利润"
                  , operate_profits
                  , is_smooth=True,
                    is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.9),
            label_opts=opts.LabelOpts(is_show=False),
                  )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=stock_info['name']),
             xaxis_opts=opts.AxisOpts(
                        is_scale=True,
                        type_="category",
                        splitarea_opts=opts.SplitAreaOpts(
                        is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                    ),
            ),
            yaxis_opts=opts.AxisOpts(
                    is_scale=True,
                    splitarea_opts=opts.SplitAreaOpts(
                        is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                    ),
            ),
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0],
                    type_="slider",
                    pos_top="90%",
                    range_start=90,
                    range_end=100,
                ),
#                 opts.DataZoomOpts(
#                     is_show=True,
#                     yaxis_index=[0],
#                     type_="slider",
#                     orient = 'vertical',
# #                     pos_top="90%",
#                     pos_right="0%",
#                     range_start=0,
#                     range_end=100,
#                 ),
            ],
            tooltip_opts=opts.TooltipOpts(
                    trigger="axis",
                    axis_pointer_type="cross",
                    background_color="rgba(245, 245, 245, 0.8)",
                    border_width=1,
                    border_color="#ccc",
                    textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
        )
    )
    return c


In [69]:
stock_keyword='000001'
start_date = '20080101'
end_date =''
stock_info ={}
stock_info['code'],stock_info['symbol'],stock_info['name'] = get_ts_code(stock_keyword)

In [81]:
pro = ts.pro_api()
df_income = pro.income(ts_code=stock_info['code'], start_date=start_date, end_date=end_date)
#df_income.to_excel("income.xlsx",sheet_name='Sheet_name_1')
df_income.index = pd.to_datetime( df_income.ann_date )
df_income = df_income.sort_index(ascending=True)


In [82]:
c = get_line_income(df_income,stock_info)
c.render_notebook()

In [86]:
df_fin = pro.fina_indicator(ts_code=stock_info['code'], start_date=start_date, end_date=end_date)
df_fin.index = pd.to_datetime( df_fin.ann_date )
df_fin = df_fin.sort_index(ascending=False)

In [89]:
df_fin[['gross_margin','netdebt','bps','profit_to_gr','saleexp_to_gr','gc_of_gr','profit_to_op','fixed_assets']]

,gross_margin,netdebt,bps,profit_to_gr,saleexp_to_gr,gc_of_gr,profit_to_op,fixed_assets
ann_date,,,,,,,,
2019-08-08,None,None,13.5658,22.7086,None,None,29.4903,1.077500e+10
2019-04-24,None,None,13.2359,22.9277,None,None,29.7789,1.082100e+10
2019-03-07,None,None,12.8182,21.2636,None,None,27.6149,1.109300e+10
2018-10-24,None,None,12.5380,23.6038,None,None,30.6540,9.568000e+09
2018-08-16,None,None,12.1251,23.3609,None,None,30.3401,8.730000e+09
2018-04-20,None,None,11.8485,23.5317,None,None,30.5680,8.076000e+09
2018-03-15,None,None,11.7706,21.9207,None,None,28.5076,8.245000e+09
2017-10-21,None,None,11.5409,23.9913,None,None,31.2390,8.095000e+09
2017-08-11,None,None,11.1532,23.2168,None,None,30.3885,8.174000e+09


In [88]:
stock_info

{'code': '000001.SZ', 'symbol': '000001', 'name': '平安银行'}

In [80]:
df_fin.ann_date

0     20190808
1     20190424
2     20190307
3     20181024
4     20180816
5     20180420
6     20180315
7     20171021
8     20170811
9     20170422
10    20170317
11    20161021
12    20160812
13    20160421
14    20160310
15    20151023
16    20150814
17    20150424
18    20150313
19    20141024
20    20140814
21    20140424
22    20140307
23    20131023
24    20130823
25    20130424
26    20130308
27    20121026
28    20120816
29    20120426
30    20120309
31    20111026
32    20110818
33    20110427
34    20110225
35    20101028
36    20100825
37    20100429
38    20100312
39    20091029
40    20090821
41    20090424
42    20090320
43    20081024
44    20080821
45    20080424
Name: ann_date, dtype: object